In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns

In [3]:
data = pd.read_csv('data-projet7.csv')
data.head()

,country,year_survey,quantile,nb_quantiles,income,gdpppp
0,ALB,2008,1,100,"728,89795",7297
1,ALB,2008,2,100,"916,66235",7297
2,ALB,2008,3,100,"1010,916",7297
3,ALB,2008,4,100,"1086,9078",7297
4,ALB,2008,5,100,"1132,6997",7297
